# Размещение баннеров

## Постановка задачи
Представим, что международное круизное агентство "Carnival Cruise Line" решило себя разрекламировать с помощью баннеров и обратилось для этого к Вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с Вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры - легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях - Foursqare.
Часть открытых данных есть, например, на сайте archive.org:
https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив с этой страницы.

## Преобразование данных
Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:

id | user_id | venue_id | latitude | longitude | created_at

---------+---------+----------+-------------------+-------------------+---------------------

984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43

...

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки не содержащие координат - они неинформативны для нас:

id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

...

С помощью pandas построим DataFrame и убедимся, что все 396632 строк с координатами считаны успешно.

In [1]:
import pandas as pd

In [6]:
lines = []
data = []

with open('checkins.dat') as input_file:
    #считаем колонки
    columns = map(lambda x: x.strip(), input_file.readline().strip().split('|'))
    n_params = len(columns)
    for line in input_file:
        items = map(lambda x: x.strip(), line.strip().split('|'))
        if len(items) == n_params:
            data.append(items)

In [3]:
df = pd.DataFrame(data)
df.columns = columns
df = df[(df.latitude != '') & (df.longitude != '')]
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824,5222,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
7,984291,105054,5222,45.5234515,-122.6762071,2012-04-21 17:39:22
9,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870,380645,33.4483771,-112.0740373,2012-04-21 17:38:18


In [7]:
df.to_csv('data.csv')

In [8]:
df.shape

(396634, 6)

# Кластеризация
Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача - хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в [sklearn user guide](http://scikit-learn.org/stable/modules/clustering.html#mean-shift), а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках, кластеризация будет работать долго (порядка часа). Для получения корректного ответа достаточно и 100000 (~2 минуты на "среднем" ноутбуке). Быть очень терпеливым не возбраняется - результат от этого только улучшится.

In [9]:
from sklearn.cluster import MeanShift

In [10]:
data = df[['latitude', 'longitude']]
data = data.applymap(float)

In [11]:
data_sample = data.sample(100000)

In [12]:
%%time
clst = MeanShift(bandwidth = 0.1)
clst.fit(data_sample)

Wall time: 5min 11s


Некоторые из получившихся кластеров содержат слишком мало точек - такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте, можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071

...

In [13]:
data_sample['cluster'] = clst.predict(data_sample)

In [14]:
clusters = pd.DataFrame(data_sample.pivot_table(index = 'cluster', aggfunc = 'count', values = 'latitude'))
clusters.columns = ['clust_size']

In [15]:
df_cluster_centers = pd.DataFrame(clst.cluster_centers_)
df_cluster_centers.columns = ['latitude_cent', 'longitude_cent']

In [18]:
df_clusters = df_cluster_centers.join(clusters)
df_clusters.to_csv('clusters.csv', index = None)
#убираем слишком маленькие кластеры
df_clusters = df_clusters[df_clusters.clust_size > 15]
df_clusters.head()

,latitude_cent,longitude_cent,clust_size
0,40.717285,-73.989358,14120
1,33.449407,-112.002611,2730
2,41.878140,-87.629811,3784
3,38.885743,-77.048918,2817
4,33.446119,-111.901731,2294


Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу "Carnival Cruise Line" адреса офисов:

 * 33.751277, -118.188740 (_Los Angeles_)
 * 25.867736, -80.324116 (_Miami_)
 * 51.503016, -0.075479 (_London_)
 * 52.378894, 4.885084 (_Amsterdam_)
 * 39.366487, 117.036146 (_Beijing_)
 * -33.868457, 151.205134 (_Sydney_)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании - широта и долгота этого центра, записанные через пробел.

In [19]:
def distance(lat1, lon1, lat2, lon2):
    return ((lat1 - lat2)**2 + (lon1 - lon2)**2) ** 0.5

carnival_cruise_line = [
    (33.751277, -118.188740),
    (25.867736, -80.324116),
    (51.503016, -0.075479),
    (52.378894, 4.885084),
    (39.366487, 117.036146),
    (-33.868457, 151.205134)
]

def caclulate_distance_to_nearest_office(lat, lon):
    min_dist = None
    for (lat_office, lon_office) in carnival_cruise_line:
        dist = distance(lat, lon, lat_office, lon_office)
        if (min_dist is None) or (dist < min_dist):
            min_dist = dist
    return min_dist

In [20]:
df_clusters['min_distance'] = map(caclulate_distance_to_nearest_office, df_clusters['latitude_cent'], df_clusters['longitude_cent'])

In [21]:
df_clusters.sort('min_distance')[:20]

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,latitude_cent,longitude_cent,clust_size,min_distance
221,-33.866668,151.209627,62,0.004836
326,52.372764,4.892524,39,0.009640
53,51.502158,-0.127408,306,0.051936
51,33.812628,-118.142636,308,0.076743
26,25.788334,-80.215562,709,0.134495
102,26.003171,-80.208801,145,0.177877
81,33.897188,-118.066691,158,0.190227
48,33.642737,-117.943830,328,0.267884
399,26.142628,-80.334694,30,0.275095
38,33.922603,-118.415075,398,0.283867


In [22]:
df_clusters.to_csv('clusters.csv', index = None)